In [1]:
import ROOT
import root_pandas
import numpy as np
import pandas as pd
from root_pandas import read_root

In [2]:
# Only load in first 50,000 events
for df in read_root('/home/shawaf/NR/mc_files/Xenon1T_ib1sp500mm_AmBe_g4mc_G4_Sort_AllNRs.root', chunksize = 2000):
    df = df
    break

In [3]:
def better_row(df_row,i):
    """ Make a better row to analyze what is ms or ss from truth data, i is the index of the row """
    ed = []
    ed.append(max(df_row.Fax_ed))
    
    s_a = np.where(df_row.Fax_ed == ed[0])
    
    s_a_x = df_row.Fax_x[s_a]
    s_a_y = df_row.Fax_y[s_a]
    s_a_z = df_row.Fax_z[s_a]
    
    if len(df_row.Fax_ed) > 1:
        ed.append(sorted(df_row.Fax_ed)[- 2])
        s_b = np.where(df_row.Fax_ed == ed[1])
        s_b_x = df_row.Fax_x[s_b]
        s_b_y = df_row.Fax_y[s_b]
        s_b_z = df_row.Fax_z[s_b]
        
        s_dist = np.sqrt((s_a_x - s_b_x)**2 + (s_a_y - s_b_y)**2 + (s_a_z - s_b_z)**2)
        
    else:
        ed.append(float('nan'))
        s_dist = float('nan')


    idx = df_row.index
    
    eventid = df_row.eventid
    
    return pd.DataFrame({
        'eventid' : eventid,
        'ed_a' : ed[0],
        'ed_b' : ed[1],
        's_dist' : s_dist
    }, index = [i])

In [4]:
def better_df(df):
    col_needed = better_row(df.iloc[0], 0)
    ndf = pd.DataFrame(columns = list(col_needed))
    for i in range(len(df)):
        row = df.iloc[i]
        br = better_row(row, i)
        ndf = ndf.append(br)
    return ndf

In [5]:
def classify_df(df):
    """ 
    takes a "better_df" ready for classification
    and classifies the events as  SS or not-SS 

    ed_a is the largest energy deposition
    ed_b is second largest energy deposition

    s_dist is the distance between these scatters

    """
    # dummy
    df['class'] = 4
    
    ed_a_lower_lim = 5.
    ed_a_upper_lim = 100.

    ed_b_lower_lim = 5.

    s_dist_lim = 200.

    for index, event in df.iterrows():
        if (
            ed_a_lower_lim < event.ed_a and
             event.ed_a < ed_a_upper_lim and 
             ed_b_lower_lim < event.ed_b and 
             event.s_dist > s_dist_lim
            ):
            df['class'][index] = 1
        else:
            df['class'][index] = 2
    return df

In [6]:
ndf = better_df(df)

In [7]:
cdf = classify_df(ndf)

/home/shawaf/anaconda3/envs/pax/lib/python3.4/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/shawaf/anaconda3/envs/pax/lib/python3.4/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
pd.read_pickle("/home/shawaf/data/mc_ambe_classify_test/mc_ambe_samp_2.pkl")

In [ ]:
import matplotlib.pyplot as plt

plt.hist(ndf.dropna().ed_b, bins = 20, normed = 1)
plt.show()

In [ ]:
plt.hist(ndf.dropna().s_dist, normed = 1, bins=40)
plt.show()

In [ ]:
# Load in sim data
import hax

# using modified version of Tianyu's OtherLargeS2s peak extractor
from make_minitree import Peaks

hax.init(experiment='XENON1T',
         use_runs_db=False,
         pax_version_policy='loose',
         main_data_paths=['/home/shawaf/data/mc_ambe_samp/'],
         minitree_paths = ['/home/shawaf/data/mc_ambe_samp/'])

In [ ]:
sim = hax.minitrees.load("mc_ambe_samp",[Peaks])

In [ ]:
sim['nFax'] = df.nFax
sim['ns'] = df.ns
sim = sim.dropna(subset=['s1','s2'])

In [ ]:
ms = []

for ns in sim.nFax:
    if ns > 1:
        ms.append(2)
    elif ns == 1:
        ms.append(1)
    else:
        ms.append(int('nan'))
        
sim['class'] = ms

In [ ]:
# Check everything is classified correctly
sim[(sim.nFax > 1) & (sim['class'] == 1)]

In [ ]:
# sim.to_pickle('full_chain_ambe_faxed')